![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [119]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("rental_info.csv", 
                      parse_dates=['rental_date', 'return_date'])
dataset.head(5)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [120]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   rental_date       15861 non-null  datetime64[ns, UTC]
 1   return_date       15861 non-null  datetime64[ns, UTC]
 2   amount            15861 non-null  float64            
 3   release_year      15861 non-null  float64            
 4   rental_rate       15861 non-null  float64            
 5   length            15861 non-null  float64            
 6   replacement_cost  15861 non-null  float64            
 7   special_features  15861 non-null  object             
 8   NC-17             15861 non-null  int64              
 9   PG                15861 non-null  int64              
 10  PG-13             15861 non-null  int64              
 11  R                 15861 non-null  int64              
 12  amount_2          15861 non-null  float64            
 13  l

In [121]:
dataset.isna().any()

rental_date         False
return_date         False
amount              False
release_year        False
rental_rate         False
length              False
replacement_cost    False
special_features    False
NC-17               False
PG                  False
PG-13               False
R                   False
amount_2            False
length_2            False
rental_rate_2       False
dtype: bool

In [122]:
dataset["rental_length"] = pd.to_datetime(dataset["return_date"]) - pd.to_datetime(dataset["rental_date"])
dataset["rental_length_days"] = dataset["rental_length"].dt.days

dataset["rental_length_days"].head(5)

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64

In [123]:
dataset["special_features"].head(5)

0    {Trailers,"Behind the Scenes"}
1    {Trailers,"Behind the Scenes"}
2    {Trailers,"Behind the Scenes"}
3    {Trailers,"Behind the Scenes"}
4    {Trailers,"Behind the Scenes"}
Name: special_features, dtype: object

In [124]:
dataset["deleted_scenes"] = dataset["special_features"].apply(lambda x: 1 if "Deleted Scenes" in x else 0)
dataset["behind_the_scenes"] = dataset["special_features"].apply(lambda x: 1 if "Behind the Scenes" in x else 0)
dataset.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

In [125]:
from sklearn.model_selection import train_test_split

cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

X = dataset.drop(cols_to_drop, axis=1)
y = dataset["rental_length_days"]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=9)

In [126]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

lasso = Lasso(alpha=0.35, random_state=9)

lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Perform feature selection by choosing columns with positive coefficients
X_lasso_train, X_lasso_test, = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [127]:
# Run OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
 
ols_model = LinearRegression()
ols_model = ols_model.fit(X_lasso_train, y_train)
y_pred = ols_model.predict(X_lasso_test)
mse_lineareg_lasso = mse(y_test, y_pred)

In [128]:
# Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}

rf = RandomForestRegressor()
rand_search = RandomizedSearchCV(rf,
                                param_distributions=param_dist,
                                cv=5,
                                random_state=9)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_rf = mse(y_test, rf_pred)

In [129]:
# Random forest gives lowest MSE so:
best_model = rf
best_mse = mse_rf

In [130]:
print(best_model)

RandomForestRegressor()


In [131]:
print(best_mse)

2.023745940985196
